In [22]:
import pandas as pd

# 建立一個空的 DataFrame
trade = pd.DataFrame(columns=['股票代號', '交易類型', '買入時間', '買入價格', '賣出時間', '賣出價格', '單位'])

# 建立一個 Series 物件表示新的行
new_trade = pd.Series(
     {'股票代號': '2330', '交易類型': 'Buy', '買入時間': '2023-01-01', '買入價格': 100, '賣出時間': '2023- 01-02', '賣出價格': 110, '單位': 1}
)

# 將 Series 轉換為 DataFrame 並進行轉置，使其成為一行
new_trade_df = new_trade.to_frame().T

# 使用 concat 方法將新的 DataFrame 物件新增至原有 DataFrame 中
trade = pd.concat([trade, new_trade_df], ignore_index=True)

# 列印新的 DataFrame
print(trade)

   股票代號 交易類型        買入時間 買入價格         賣出時間 賣出價格 單位
0  2330  Buy  2023-01-01  100  2023- 01-02  110  1


在新版本 pandas 使用 append 時要改用 _append

In [21]:
import pandas as pd

# 建立一個空的 DataFrame
trade = pd.DataFrame(columns=['股票代號', '交易類型', '買入時間', '買入價格', '賣出時間', '賣出價格', '單位'])

# 建立一個 Series 物件表示新的行
new_trade = pd.Series(
     {'股票代號': '2330', '交易類型': 'Buy', '買入時間': '2023-01-01', '買入價格': 100, '賣出時間': '2023- 01-02', '賣出價格': 110, '單位': 1}
)

# 使用 append 方法將新的 Series 物件新增至 DataFrame 中
trade = trade._append(new_trade, ignore_index=True)

# 列印新的 DataFrame
print(trade)

   股票代號 交易類型        買入時間 買入價格         賣出時間 賣出價格 單位
0  2330  Buy  2023-01-01  100  2023- 01-02  110  1


In [ ]:
import yfinance as yf
import os
import pandas as pd
import mplfinance as mpf


# 更新資料源為 yahoo finance
def getDataFM(prod, st, en):
    # 下載檔案
    _file = "data//YF_%s_%s_%s_stock_daily_adj.csv" % (prod, st, en)
    if os.path.exists(_file):
        data = pd.read_csv(_file)
        data["Date"] = pd.to_datetime(data["Date"])
        data = data.set_index("Date")
    else:
        data = yf.download(f"{prod}.TW", start=st, end=en)
        data.columns = [i.lower() for i in data.columns]
        # 除錯 如果沒有資料
        if data.shape[0] == 0:
            print("沒有資料")
            return pd.DataFrame()
        # 將資料寫入備份檔
        data.to_csv(_file)
    return data

In [ ]:
# 取得回測資料
prod = "2330"
data = getDataFM(prod, "2013-01-01", "2023-12-15")

In [ ]:
# 繪製交易記錄圖
def ChartTrade(data, trade=pd.DataFrame(), addp=[], v_enable=True):
    addp = addp.copy()
    data1 = data.copy()
    # 如果有交易紀錄，則把交易紀錄與K線彙整
    if trade.shape[0] > 0:
        # 將物件複製出來，不影響原本的交易明細變數
        trade1 = trade.copy()
        # 取出進場明細，透過時間索引將資料合併
        buy_order_trade = trade1[[2, 3]]
        buy_order_trade = buy_order_trade.set_index(2)
        buy_order_trade.columns = ["buy_order"]
        buy_order_trade = buy_order_trade.drop_duplicates()
        # 取出出場明細，透過時間索引將資料合併
        buy_cover_trade = trade1[[4, 5]]
        buy_cover_trade = buy_cover_trade.set_index(4)
        buy_cover_trade.columns = ["buy_cover"]
        buy_cover_trade = buy_cover_trade.drop_duplicates()
        # 將交易紀錄與K線資料彙整
        data1 = pd.concat([data1, buy_order_trade, buy_cover_trade], axis=1)

        # 將交易紀錄透過副圖的方式繪製
        addp.append(
            mpf.make_addplot(
                data1["buy_order"],
                type="scatter",
                color="#FF4500",
                marker="^",
                markersize=50,
            )
        )
        addp.append(
            mpf.make_addplot(
                data1["buy_cover"],
                type="scatter",
                color="#16982B",
                marker="v",
                markersize=50,
            )
        )
    # 繪製圖表
    mcolor = mpf.make_marketcolors(up="r", down="g", inherit=True)
    mstyle = mpf.make_mpf_style(base_mpf_style="yahoo", marketcolors=mcolor)
    mpf.plot(data1, addplot=addp, style=mstyle, type="candle", volume=v_enable)


In [ ]:
# 初始部位
position = 0
trade = pd.DataFrame()
print("***********************")
print(type(trade))

In [ ]:
# 開始回測
for i in range(data.shape[0] - 1):
    # 取得策略會應用到的變數
    c_time = data.index[i]
    c_low = data.loc[c_time, "low"]
    c_high = data.loc[c_time, "high"]
    c_close = data.loc[c_time, "close"]
    c_open = data.loc[c_time, "open"]
    # 取下一期資料做為進場資料
    n_time = data.index[i + 1]
    n_open = data.loc[n_time, "open"]

    # 進場程序
    if position == 0:
        # 進場邏輯
        if c_close > c_open and (c_close - c_open) * 2 < (c_open - c_low):
            position = 1
            order_i = i
            order_time = n_time
            order_price = n_open
            order_unit = 1
    # 出場程序
    elif position == 1:
        # 出場邏輯
        if i > order_i + 3 and c_close > c_open:
            position = 0
            cover_time = n_time
            cover_price = n_open
            # 交易紀錄
            # trade = trade._append(
            trade = trade.concat(
                pd.Series(
                    [
                        prod,
                        "Buy",
                        order_time,
                        order_price,
                        cover_time,
                        cover_price,
                        order_unit,
                    ]
                ),
                ignore_index=True,
            )


# 繪製K線圖與交易明細
ChartTrade(data, trade)